# Calibrating the Zahn Cup using a Viscosity Standard

The Zahn cup that we'll use in our viscosity lab was, unfortunately, very poorly calibrated by the manufacturer.  The good news: this gives us a chance to calibrate an instrument that badly needs to be calibrated!

<br>

<center>
<img src = https://github.com/MAugspurger/Exper_Eng/raw/main/Labs_and_Sensors/Images/wholesom.PNG width = 300>
</center>

<br>

To do this, we'll follow a set of steps:
1.  Take



## Fitting and Plotting a Curve in Python

If we just want a simple linear fit, we can use a spreadsheet.  Sometimes, though, we might want to fit a curve to data that is not governed by a linear, polynomial or expontial equation.  

<br> This is the case with the viscosity/ temperature relationship, which is often expressed as an Arrhenius equation.  This equation has the form:

$$\mu = \mu_0 e^{B/T} $$

<br> where B is a constant related to intermolecular energies that control viscosity and $\mu_0$ is a constant that defines a limit as the temperature $T$ gets very high. The temperature $T$ must be in Kelvin.

<br> In this notebook, we'll use this equation to fit a curve to data for the viscosity of water.

In [ ]:
import pandas as pd
import numpy as np

# Create arrays for temperature, viscosity, and density
temp_C = np.array([20.00, 25.00, 37.78, 40.00, 50.00])#, 80.00]) #98.89, 100.00])
temp_F = temp_C*1.8+32.0
N35_cP = np.array([74.60, 56.39, 30.18, 27.34, 18.25]) #, 7.104]) #, 4.551, 4.445])
density_N35 = np.array([0.8619, 0.8588, 0.8507, 0.8493, 0.8430]) #, 0.8241]) #0.8123, 0.8116])
N35_cSt = np.divide(N35_cP,density_N35)

# Create series in cP (dynamic/ absolute vis) and cST (kinematic viscosity)
N35_C_cP = pd.Series(data=N35_cP, index=temp_C)
N35_C_cSt = pd.Series(data=N35_cSt, index=temp_C)
N35_F_cP = pd.Series(data=N35_cP, index=temp_F)
N35_F_cSt = pd.Series(data=N35_cSt, index=temp_F)

### Defining the Arrhenius curve

Here's a function to create a plot of an Arrhenius curve.  `T_array` will be an array of our x-axis values: that is, the temperature $T$ values for which we will be defining a viscosity.  


In [ ]:
def arrhenius(mu_0,B,T_array):
    mu = mu_0 * np.exp(B/(T_array+273.15))
    return pd.Series(index = T_array,data=mu)

### Using `leastsq` to find the best fit

The shape of our curve is determined by the values of $B$ and $\mu_0$.  We want to find the values for these parameters that fits the data, and we'll use the least squares method to do that.  Python, fortunately, can save us a lot of trouble.

<br> Remember that to use least squares, we need to define an error function: a function that finds the difference between the curve and the data at each point.  This difference is written as $y_i - y_c$ in our class notes about the standard error of the fit.

In [ ]:
def error_func(params, data):
    arrh = arrhenius(params[0],params[1],data.index)
    errors = arrh - data
    return errors


Notice that we put in `data.index` as our `T_array`: this is because we need `arrh` and `data` defined at the same temperatures so that we can find the difference between them in line 3.  

<br> We can run this with some estimated parameters:

In [ ]:
params = [0.00005,4200]
arrh = arrhenius(params[0],params[1],N35_C_cSt.index)
N35_C_cSt.plot(ylabel = 'viscosity (cP)', xlabel = 'Temperature (C)',
           title = 'Viscosity of N35 Standard', style = 'o',legend=True);
arrh.plot()
error_func(params,N35_C_cSt)

First, look at this plot and convince yourself that the list of errors matches with the plot.  If that seems in order, run `leastsq`, which is a SciPy algorithm that will minimize the error function we just wrote:

In [ ]:
import scipy.optimize as spo
best_params, fit_details = spo.leastsq(error_func, params, N35_C_cSt)


Now we'll plot the fitted curve with the known data:

In [ ]:
temp_range = np.linspace(20.0,50.0,31)
arrh_best = arrhenius(best_params[0],best_params[1],temp_range)
N35_C_cSt.plot(ylabel = 'viscosity (cP)', xlabel = 'Temperature (C)',
           title = 'Viscosity of N35 Standard', style = 'o', label='Data',legend=True);
arrh_best.plot(label='Fitted Arrhenius', legend =True);
print("mu_0 = ", best_params[0].round(9), " B = ", best_params[1].round(1))



In [ ]:
T_one = np.array([19.0])
arrhenius(0.000030996, 4347.0, T_one)

### Finding Zahn cup calibration equation

We now know that the N35 standard is:

<br>

$$\mu = 0.00001898 e^{4492.8/T} $$

<br>

The form of the conversion for the Zahn cup is:

<br>

$$\mu = K(t - c)$$

<br>

where $K$ and $c$ are fitted coefficients.  You have time data for the zahn cups, and we want to find the two coefficients that make this curve as close as possible to the curve for the standard.


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import pandas as pd
import numpy as np

home = '/gdrive/My Drive/Teaching/Engr_290/Notebooks_290/Curve_fitting/'
filename = 'Zahn_times_standard_N35'
file = home + filename + ".xlsx"
vis_data = pd.read_excel(file)

In [ ]:
# First enter your data as an array
zahn_times1 = vis_data.Time1.values
temps1 = vis_data.Temp1.values
zahn_times2 = vis_data.Time2.values
temps2 = vis_data.Temp2.values

In [ ]:
# Find the viscosity for the standard for these temps
mu_0 = best_params[0]
B = best_params[1]
stand1_visc = mu_0 * np.exp(B/(temps1+273.15))
stand1 = pd.Series(index = temps1, data = stand1_visc)

In [ ]:
# Now create a function with the coefficients K and c to calculate viscosity
def zahn(K,c,t_array, temps):
    mu = K*(t_array - c)
    return pd.Series(index = temps,data=mu)

In [ ]:
# Test the function with the Manufacturer's numbers
mu_zahn = zahn(1.1, 29, zahn_times1, temps1)
mu_zahn.plot(ylabel = 'viscosity (cSt)', xlabel = 'Temperature (C)',
           title = 'Viscosity of N35 Standard (Zahn Cup)', style = 'o',
             legend=True, label="Predicted Viscosity using Zahn Cup")
#stand1.plot(label="Known Viscosity of Standard N35", legend=True);
arrh_best.plot(label="Known Viscosity of Standard N35", legend=True);

Not a very good fit (what's up with the manufacturer here?  A good rule of them with inexpensive Chinese lab equipment: "trust but calibrate").  Now we need to find a new $K$ and $c$ that will minimize this difference.

In [ ]:
# Make function that calculates the difference between the two sets of values
def error_func(params, t_vals, standard_vals):
    zahn_vals = zahn(params[0], params[1], t_vals, standard_vals.index)
    errors = zahn_vals - standard_vals.values
    return errors

In [ ]:
# Test the error function
params = np.array([1.1, 29])
error_func(params, zahn_times1, stand1)

In [ ]:
# Now minimize the difference between the Zahn measurements and the standard
best_K_c, fit_details = spo.leastsq(error_func, params, args = (zahn_times1, stand1))
best_K_c

In [ ]:
# Plot the zahn cup values with these best parameters
mu_zahn = zahn(best_K_c[0], best_K_c[1], zahn_times1, temps1)
mu_zahn.plot(ylabel = 'viscosity (cSt)', xlabel = 'Temperature (C)',
           title = 'Viscosity of N35 Standard (Zahn Cup #1)',
             label='Zahn Cup Values with Best Coefficients',
             legend=True, style='o')
stand1.plot(label='Known Values of Standard',legend=True, style = 'o');

In [ ]:
# Now solve this for the #2 Zahn cup
# Find the viscosity for the standard for these temps
mu_0 = best_params[0]
B = best_params[1]
stand2_array = mu_0 * np.exp(B/(temps2+273.15))
stand2 = pd.Series(index = temps2, data = stand2_array)

# Now minimize the difference between the Zahn measurements and the standard
best_K_c2, fit_details = spo.leastsq(error_func, params, args = (zahn_times2, stand2))
best_K_c2

In [ ]:
# Plot the zahn cup values with these best parameters
mu_zahn2 = zahn(best_K_c2[0], best_K_c2[1], zahn_times2, temps2)
mu_zahn2.plot(ylabel = 'viscosity (cSt)', xlabel = 'Temperature (C)',
           title = 'Viscosity of N35 Standard (Zahn Cup #2)',
             label='Zahn Cup Values with Best Coefficients',
             legend=True, style = 'x')
stand2.plot(label='Known Values of Standard',legend=True, style = 'o');

### Finding the standard error of the fit

Now you can do a little work 😀  Look at the equation for the standard error of the fit in the class notes, and calculate that for our model.  Follow each step, and print out your answer for each step to make sure it makes sense:

In [ ]:
# Make an array of y_i - y_c for each known value
# The values in this array should match the errors printed above
error_array = ?????????
error_array

# Now square each point in the error_array
#sqerr_array = ????????

# Find the sum of the squared errors
# You may need to look up how to find the sum of a Series
#sum_err =  ????????

# Find nu (assume that the order of the fit m = 2)
#nu =   ????????

# Divide sum_err by nu and take the square root
#s_yx =  ????????


Your standard error of the fit for this data `s_yx` should be about 0.013 cP.

### Plotting with error bars

The standard error of the fit does *not* tell us the error in each data point.  Instead, it tells us the potential error in the fit: how far off the fit might be.   So we want to draw our error bars not from the data points, but from the curve itself.

<br> We can do this using standard error bars.  Notice that we've only added one keyword argument (`yerr`) to this plot command:

In [ ]:
water.plot(ylabel = 'viscosity (cP)', xlabel = 'Temperature (C)',
           title = 'Viscosity of Water', style = 'o', label='Data',legend=True);
arrh_best.plot(label='Fitted Arrhenius', legend =True, yerr = s_yx);

But since the error is in the curve, it actually more sense to show a "bounds" for the entire curve.  We can do that using MatPlotLib, which is a powerful tool for plotting in Python (and in fact is the code on which our Series and DataFrame plot() functions are built).

<br> `fill_between` defines an upper and lower bound for the curve, and shades in the region.  `alpha` defines the transparency of the shaded region.

In [ ]:
import matplotlib.pyplot as plt
water.plot(ylabel = 'viscosity (cP)', xlabel = 'Temperature (C)',
           title = 'Viscosity of Water', style = 'o', label='Data',legend=True);
arrh_best.plot(label='Fitted Arrhenius', legend =True);
plt.fill_between(arrh_best.index, arrh_best.values - s_yx, arrh_best.values + s_yx,
                 color = 'gray', alpha = 0.3);

So now that you have a model for this type of plot, you can use it to display your own viscosity data.  Yippee!

### Exercise

✅ ✅ Create a plot showing the error bounds for a fitted Arrhenius curve for the Ethenol data (which is part of the spreadsheet data we imported at the top of this notebook).   Notice that you will not need to do a lot of coding to do this: recognize what tools we've already created and use those tools!